In [1]:
import pandas as pd
import numpy as np
import os
import requests

In [43]:
# pip install mysql-connector-python

In [42]:
import pyodbc
import urllib
import sqlalchemy
# MySQL connection parameters
db_params = {
    'user': 'han',
    'password': 'joseph',
    'host': '127.0.0.1',  # Usually 'localhost' or an IP address
    'port': '4450',  # Default MySQL port is 3306
    'database': 'ITRAVEL',
}

# URL-encode the password (if it contains special characters)
db_params['password'] = urllib.parse.quote_plus(db_params['password'])

# Create a MySQL database connection using SQLAlchemy
params_mysql = "mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}".format(**db_params)
engine_mysql = sqlalchemy.create_engine(params_mysql)

In [43]:
engine_p=engine_mysql

In [44]:
'''get locations data from source database'''
df_location_existing = pd.read_sql_table('locations',engine_p)

In [45]:
df_location_existing

,id,location_id,name,distance,bearing,address_obj
0,1,15706617,Bib - English Breakfast in the Box,9.540954320756208,southwest,"{'street1': 'Lower Ground Level, Al Shafar Tow..."
1,2,10165445,Ayamna,10.884259167826489,southwest,"{'street1': 'Crescent road', 'street2': 'The P..."
2,3,1022759,Atlantis The Palm,10.907456424513168,southwest,"{'street1': 'Crescent Road', 'city': 'Dubai', ..."
3,4,17836852,Tulum Restaurant,0.6873599138082985,southeast,"{'street1': 'The Dubai Mall Unit No. 4F - 04',..."
4,5,942786,Asado Restaurant,0.8196275775014424,southeast,"{'street1': 'Shaik Rashid Blvd', 'street2': 'T..."
...,...,...,...,...,...,...
2022,2023,23131424,Luxurious 2BR in Palm - Beach Access & 2 Balco...,0.8584219616280413,south,"{'street1': '', 'street2': '', 'city': 'Hatta'..."
2023,2024,306743,Palm Beach Hotel,3.924626870515377,north,"{'street1': 'Khalid Bin Al Waleed Rd 26', 'str..."
2024,2025,21085672,Penthouse with Burj Khalifa View in Address Sk...,0.2964937614919032,south,"{'street1': '', 'street2': '', 'city': 'Dubai'..."
2025,2026,26711205,Long's Bar - Towers Rotana,0.62870235457428,northeast,"{'street1': 'Towers Rotana', 'street2': 'Sheik..."


In [46]:
# Define a location reviews function
def call_locations(location):
    """This function get location reviews through api ."""
    
    # API URL
    api_url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=" + str(location) + "&latLong=25.2048%2C%2055.2708&language=en&key=72577774A72F48D9BF1180DEA12E312A"

    # Make API request
    response = requests.get(api_url)
    data = response.json()
    
    return data 

In [2]:
'''read hotels name from local file'''

import re
# Define a list of file paths
file_paths = ['F:\itravel\data\hotels_1_20\hotels_11' ,'F:\itravel\data\hotels_1_20\hotels_12' , 
             'F:\itravel\data\hotels_1_20\hotels_13' ,'F:\itravel\data\hotels_1_20\hotels_14' ,
             'F:\itravel\data\hotels_1_20\hotels_15' ,'F:\itravel\data\hotels_1_20\hotels_16' ,
             'F:\itravel\data\hotels_1_20\hotels_17' ,'F:\itravel\data\hotels_1_20\hotels_18' ,
             'F:\itravel\data\hotels_1_20\hotels_19' ,'F:\itravel\data\hotels_1_20\hotels_20' ]

data=''
# Loop through the file paths and read each file into a DataFrame
for file_path in file_paths:
    
    # file_path = 'C:/Users/pc/Downloads/restaurant_16'

    # Open the file in read mode ('r')
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read the entire file content into a string
        data = data + file.read()
        
# Use regular expressions to find lines that start with a number
line_numbers = re.findall(r'^\d+\..*', data, re.MULTILINE)


# Remove serial numbers using regular expression
cleaned_strings = [ re.sub(r'^\d+\.\s+', '', s) for s in line_numbers]

# Remove empty strings
non_empty_strings = [s for s in cleaned_strings if s]

# Remove duplicates by converting the list to a set and back to a list
non_empty_strings = list(set(non_empty_strings))

non_empty_strings


['President Hotel',
 'Mena ApartHotel Al Barsha',
 'Al Manar Hotel Apartments',
 'City Avenue Hotel',
 'Dubai Marriott Harbour Hotel & Suites',
 'Admiral Plaza Hotel',
 'Golden Sands Hotel Creek',
 'Aravi Hotel',
 'Edge Creekside Hotel',
 'Flora Creek Deluxe Hotel Apartments',
 'The Ritz-Carlton, Dubai International Financial Centre',
 "Jumeirah Mina A'Salam",
 'Staybridge Suites Dubai Al-Maktoum Airport, an IHG hotel',
 'Sheraton Jumeirah Beach Resort',
 'TIME Grand Plaza Hotel',
 'Panorama Hotel Bur Dubai',
 'Pullman Dubai Jumeirah Lakes Towers',
 'Ibis World Trade Centre Dubai',
 'Mount Royal Hotel',
 'La Ville Hotel & Suites CITY WALK Dubai, Autograph Collection',
 'Hyatt Place Dubai Baniyas Square',
 'City Palace Hotel',
 'Novotel World Trade Centre Dubai',
 'Baity Hotel Apartments',
 'Dubai Grand Hotel By Fortune',
 'Bulgari Hotel & Resorts, Dubai',
 'Crowne Plaza Dubai Jumeirah',
 "Sleep 'n fly at Dubai International Airport B-Gates /Terminal 3",
 'Centro Barsha',
 'Unlock Metro

In [47]:
# Loop through DataFrame rows
df_locations_new=pd.DataFrame()

for item in non_empty_strings:
    location = item
    data=call_locations(location)
#     df1 = pd.DataFrame(data['data'])
            
    try:
        df1 = pd.DataFrame(data['data'])
    except KeyError:
        print("Key 'data' not found in the JSON response.")
        df1 = pd.DataFrame() 
    
    df_locations_new=pd.concat([df_locations_new, df1], ignore_index=True)
    
df_locations_new


,location_id,name,distance,bearing,address_obj
0,14800853,Cora Spa Dubai,0.7812907643561066,northeast,"{'street1': 'Segunda de Palomares 349', 'stree..."
1,8756403,Sofitel Spa Dubai Downtown,0.3924810925313583,south,"{'street1': 'Sheikh Zayed Road', 'street2': 'S..."
2,5415101,Hotel Sofitel Dubai Downtown,0.34928873981323555,south,"{'street1': 'Sheikh Zayed Rd', 'city': 'Dubai'..."
3,572764,Swissotel Al Murooj Dubai,0.39768086665057717,east,"{'street1': 'Al Mustaqbal Steet', 'street2': '..."
4,301379,Dusit Thani Dubai,0.16593820112373253,northeast,"{'street1': '133 Sheikh Zayed Road', 'street2'..."
...,...,...,...,...,...
2980,10818415,Carnival by Tresind,0.8355224318007782,northeast,"{'street1': '66 312 Rd', 'street2': 'Unit P5-0..."
2981,25175513,La Nina Dubai,0.7768789925801154,northeast,"{'street1': 'ICD Brookfield', 'street2': 'Firs..."
2982,1025733,"Four Points by Sheraton Sheikh Zayed Road, Dubai",0.722230174105125,northeast,"{'street1': 'Sheikh Zayed Road', 'city': 'Duba..."
2983,23265217,Residence Inn By Marriott Sheikh Zayed Road,0.42481377033807366,northeast,"{'street1': 'Sheikh Zayed Rd', 'street2': 'Tra..."


In [48]:
df_locations_new.info()
df_locations_new['location_id'] = df_locations_new['location_id'].astype(int)
df_locations_new['address_obj'] = df_locations_new['address_obj'].astype(str)
# df_locations_new['category']='Restaurant'
df_locations_new=df_locations_new.drop_duplicates(subset='location_id')
df_locations_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985 entries, 0 to 2984
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  2985 non-null   object
 1   name         2985 non-null   object
 2   distance     2985 non-null   object
 3   bearing      2985 non-null   object
 4   address_obj  2985 non-null   object
dtypes: object(5)
memory usage: 116.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 0 to 2981
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  474 non-null    int32 
 1   name         474 non-null    object
 2   distance     474 non-null    object
 3   bearing      474 non-null    object
 4   address_obj  474 non-null    object
dtypes: int32(1), object(4)
memory usage: 20.4+ KB


In [50]:
df_locations_new=df_locations_new[~df_locations_new['location_id'].isin(df_location_existing['location_id'])]

df_locations_new = df_locations_new.reset_index(drop=True)

In [51]:
df_locations_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  124 non-null    int32 
 1   name         124 non-null    object
 2   distance     124 non-null    object
 3   bearing      124 non-null    object
 4   address_obj  124 non-null    object
dtypes: int32(1), object(4)
memory usage: 4.5+ KB


In [52]:
# Save DataFrame to SQL database
# df.to_sql("Locations",engine_p,if_exists='append',index=False)
df_locations_new.to_sql("locations",engine_mysql,if_exists='append',index=False)


# print("Data has been retrieved and saved to the database.")

124